## 1. Keyword/Text Search - scoring

- custom score: https://learn.microsoft.com/en-us/azure/search/index-add-scoring-profiles
- https://learn.microsoft.com/en-us/python/api/azure-search-documents/azure.search.documents.indexes.models.searchindex?view=azure-python

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration, 
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    ScoringProfile,
    TextWeights
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

https://ikaisch.search.windows.net


In [32]:
# Create a search index
def create_search_index(index_name):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True) 
    ]

    custom_scores = [
        ScoringProfile(
            name="custom-score1",
            text_weights=TextWeights(weights={
                "proverb": 1,
                "meaning": 9
            })
        ),
        ScoringProfile(
            name="custom-score2",
            text_weights=TextWeights(weights={
                "proverb": 5,
                "meaning": 5
            })
        )
    ]

    semantic_config = SemanticConfiguration(  
        name="semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(  
            content_fields=[SemanticField(field_name="meaning")]  
        )
    )

    # Create the semantic search with the configuration  
    semantic_search = SemanticSearch(configurations=[semantic_config]) 

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields,
                        semantic_search=semantic_search,
                        scoring_profiles=custom_scores,
                        default_scoring_profile="custom-score1")
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

In [33]:
index_name = "korproverb-score-index"
create_search_index(index_name)

 korproverb-score-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [10]:
df = pd.read_csv('./source/idioms.csv')
df.head()

,Description,Meaning,Source,Type
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,속담
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,속담
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,속담
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,속담
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,속담


In [11]:
count = 0
batch_size = 40
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'proverb': row['Description'], 
                     'meaning': row['Meaning'],
                     'source': row['Source'],
                     'type': str(row['Type'])
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 164/164 [00:27<00:00,  6.05it/s]


### Semantic rerank

combination of `title(description)` text + `meaning` text search result

In [12]:
from azure.search.documents.models import (
    QueryType,
    QueryCaptionType,
    QueryAnswerType)

In [44]:
def azsch_rerank_query(query, scoring_profile="custom-score1"):
    results = search_client.search(  
        search_text=query,
        select=["proverb", "meaning", "source", "type"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='semantic-config',
        #query_caption=QueryCaptionType.EXTRACTIVE,
        #query_answer=QueryAnswerType.EXTRACTIVE,
        scoring_profile=scoring_profile,
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for result in results:  
        print(f"{result['@search.reranker_score']:.5f}/{result['@search.score']:.5f}: {result['proverb']} - {result['meaning']}, {result['source']}")   
 
def azsch_textscore_query(query, scoring_profile="custom-score1"):
    results = search_client.search(  
        search_text=query,
        select=["proverb", "meaning", "source", "type"],
        scoring_profile=scoring_profile,
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for result in results:  
        print(f"{result['@search.score']:.5f}: {result['proverb']} - {result['meaning']}, {result['source']}")  


In [39]:
azsch_rerank_query('열심히 노력하면 성공한다', scoring_profile="custom-score1")

2.47481/38.23729: 소같이 벌어서 쥐같이 먹어라 - 소같이 꾸준하고 힘써 일하여 많이 벌어서는 쥐같이 조금씩 먹으라는 뜻으로, 일은 열심히 하여서 돈은 많이 벌고 생활은 아껴서 검소하게 하라는 말., 표준국어대사전
2.46595/111.33922: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
2.45425/55.11280: 골나면 보리방아 더 잘 찧는다〔찧듯 한다〕 - 골나면 화가 치밀어 올라 그것을 해소하느라고 오히려 일을 더 힘차고 열심히 하게 된다는 말., 고려대 한국어대사전
2.43440/130.47287: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 고려대 한국어대사전
2.41703/38.23729: 가는 말에 채찍질 - 열심히 하고 있는데도 더 빨리하라고 독촉함을 비유적으로 이르는 말.,형편이나 힘이 한창 좋을 때라도 더욱 마음을 써서 힘써야 함을 비유적으로 이르는 말., 표준국어대사전
2.39293/52.40345: 깊은 물이라야 큰 고기가 논다 - 깊은 물에 큰 고기가 놀듯이 포부가 큰 사람이라야 큰일을 이루고 성공을 하게 된다는 말., 고려대 한국어대사전
2.39080/50.00365: 구르는 돌은 이끼가 안 낀다 - 부지런하고 꾸준히 노력하는 사람은 침체되지 않고 계속 발전한다는 말., 표준국어대사전
2.36971/60.38869: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전
2.36191/37.57111: 굼벵이도 제 일 하려면 한 길은 판다 - 미련한 사람이나 하찮은 미물이라도 제가 필요하면 얼마든지 나름대로 노력을 하게 된다는 말. (=굼벵이도 제 일 하는 날은 열 번 재주를 넘는다.), 고려대 한국어대사전
2.35518/64.21277: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 표준국어대사전


In [40]:
azsch_rerank_query('열심히 노력하면 성공한다', scoring_profile="custom-score2")

2.47481/21.24294: 소같이 벌어서 쥐같이 먹어라 - 소같이 꾸준하고 힘써 일하여 많이 벌어서는 쥐같이 조금씩 먹으라는 뜻으로, 일은 열심히 하여서 돈은 많이 벌고 생활은 아껴서 검소하게 하라는 말., 표준국어대사전
2.46595/86.46639: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
2.45425/30.61822: 골나면 보리방아 더 잘 찧는다〔찧듯 한다〕 - 골나면 화가 치밀어 올라 그것을 해소하느라고 오히려 일을 더 힘차고 열심히 하게 된다는 말., 고려대 한국어대사전
2.43440/72.48492: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 고려대 한국어대사전
2.41703/21.24294: 가는 말에 채찍질 - 열심히 하고 있는데도 더 빨리하라고 독촉함을 비유적으로 이르는 말.,형편이나 힘이 한창 좋을 때라도 더욱 마음을 써서 힘써야 함을 비유적으로 이르는 말., 표준국어대사전
2.39293/29.11303: 깊은 물이라야 큰 고기가 논다 - 깊은 물에 큰 고기가 놀듯이 포부가 큰 사람이라야 큰일을 이루고 성공을 하게 된다는 말., 고려대 한국어대사전
2.39080/27.77980: 구르는 돌은 이끼가 안 낀다 - 부지런하고 꾸준히 노력하는 사람은 침체되지 않고 계속 발전한다는 말., 표준국어대사전
2.36971/33.54927: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전
2.36191/20.87283: 굼벵이도 제 일 하려면 한 길은 판다 - 미련한 사람이나 하찮은 미물이라도 제가 필요하면 얼마든지 나름대로 노력을 하게 된다는 말. (=굼벵이도 제 일 하는 날은 열 번 재주를 넘는다.), 고려대 한국어대사전
2.35518/35.67376: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 표준국어대사전


In [45]:
azsch_textscore_query('열심히 노력하면 성공한다', scoring_profile="custom-score1")

130.47287: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 고려대 한국어대사전
111.33922: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
71.23980: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 고려대 한국어대사전
69.54827: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 노력하면 큰일을 할 수 있다는 말., 고려대 한국어대사전
68.10518: 눈과 귀가 쏠리다 - 마음이 끌리어 열심히 듣거나 보다., 표준국어대사전
64.21277: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 표준국어대사전
63.26539: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 고려대 한국어대사전
60.83084: 소매가 길면 춤을 잘 추고 돈이 많으면 장사를 잘한다 - 수단이나 밑천이 든든하면 성공하기 쉽다는 말., 표준국어대사전
60.71473: 힘을 돌리다 - 어떤 일을 이루려고 거기에 노력을 들이다., 표준국어대사전
60.38869: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전


In [46]:
azsch_textscore_query('열심히 노력하면 성공한다', scoring_profile="custom-score2")

86.46639: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
72.48492: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 고려대 한국어대사전
39.57767: 몸부림 치다 - (사람이) 온갖 노력을 다하다., 고려대 한국어대사전
38.63793: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 노력하면 큰일을 할 수 있다는 말., 고려대 한국어대사전
37.83621: 눈과 귀가 쏠리다 - 마음이 끌리어 열심히 듣거나 보다., 표준국어대사전
35.67376: 피땀 흘리다 - 온갖 힘과 정성을 쏟아 노력하다., 표준국어대사전
35.14744: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 고려대 한국어대사전
33.79491: 소매가 길면 춤을 잘 추고 돈이 많으면 장사를 잘한다 - 수단이나 밑천이 든든하면 성공하기 쉽다는 말., 표준국어대사전
33.73040: 힘을 돌리다 - 어떤 일을 이루려고 거기에 노력을 들이다., 표준국어대사전
33.54927: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전
